In [1]:
import pandas as pd

df = pd.read_csv('research_traffic.csv')
df.shape

(140000, 12)

In [2]:
print(df.dtypes)
print(df['Label'].value_counts()[1])
print(df['Label'].value_counts()[0])
df.head()

Count_of_Source_IP        int64
Port_Count                int64
Pair_Count_Ratio        float64
Packet_Count_Diff         int64
Lookup_Count_Diff         int64
Protocol                  int64
Average_Packet_Count    float64
Average_Byte_Count      float64
Packet_Std_Dev          float64
Byte_Std_Dev            float64
Duration_per_Flow       float64
Label                     int64
dtype: object
70000
70000


,Count_of_Source_IP,Port_Count,Pair_Count_Ratio,Packet_Count_Diff,Lookup_Count_Diff,Protocol,Average_Packet_Count,Average_Byte_Count,Packet_Std_Dev,Byte_Std_Dev,Duration_per_Flow,Label
0,2,2,1.0,436913,1,6,145637.6667,3.567915e+09,24719.5,5.339086e+09,1.258000,0
1,2,2,1.0,436912,1,6,145637.3333,3.567915e+09,24721.0,5.339086e+09,1.254000,0
2,2,2,1.0,1006277,1,6,335425.6667,8.291520e+09,61289.5,1.240812e+10,2.587333,0
3,2,2,0.0,1,1006280,6,335426.0000,8.291520e+09,61288.0,1.240812e+10,2.592000,0
4,2,2,1.0,1605562,6,6,535187.3333,1.321503e+10,97006.0,1.977597e+10,3.926000,0


In [3]:
# obj_cols=df.dtypes[df.dtypes == "object"].index.values.tolist()
# print(obj_cols)

# from sklearn.preprocessing import LabelEncoder
# #Encode labels of multiple columns at once

# df[obj_cols] = df[obj_cols].astype(str)
# df[obj_cols] = df[obj_cols].apply(LabelEncoder().fit_transform)
# #
# # Print head
# #
# print(df.dtypes)
# df.head()

In [4]:
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from datetime import datetime

y = df.Label
X = df.drop(['Label'],axis=1)

# Normalize the dataset using MinMaxScaler
normalizer = MinMaxScaler()
X_normalized = normalizer.fit_transform(X)
X = pd.DataFrame(X_normalized, columns=X.columns)
print("Dataset normalized using MinMaxScaler.")

X_train_cv, X_unseen_test, y_train_cv, y_unseen_test = train_test_split(X, y, test_size=0.2, random_state=42)

kf = KFold(n_splits=10, shuffle=True, random_state=42)

Dataset normalized using MinMaxScaler.


In [5]:
def create_model():
    model = Sequential()
    model.add(Dense(32, input_dim=X_train_cv.shape[1], activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [6]:
# Initialize lists to store metrics
train_accuracies = []
test_accuracies = []
precisions = []
recalls = []
f1_scores = []
confusion_matrices = []
start = datetime.now()

for train_index, test_index in kf.split(X_train_cv):
    X_train, X_test = X_train_cv.iloc[train_index], X_train_cv.iloc[test_index]
    y_train, y_test = y_train_cv.iloc[train_index], y_train_cv.iloc[test_index]
    
    model = create_model()
    model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0)
    
    # Evaluate on training data
    y_pred_train = (model.predict(X_train) > 0.5).astype(int)
    y_pred_test = (model.predict(X_test) > 0.5).astype(int)
    
    # Calculate metrics for the test fold
    train_accuracies.append(accuracy_score(y_train, y_pred_train))
    test_accuracies.append(accuracy_score(y_test, y_pred_test))
    precisions.append(precision_score(y_test, y_pred_test))
    recalls.append(recall_score(y_test, y_pred_test))
    f1_scores.append(f1_score(y_test, y_pred_test))
    confusion_matrices.append(confusion_matrix(y_test, y_pred_test))

y_unseen_pred = (model.predict(X_unseen_test) > 0.5).astype(int)
unseen_accuracy = accuracy_score(y_unseen_test, y_unseen_pred)
unseen_precision = precision_score(y_unseen_test, y_unseen_pred)
unseen_recall = recall_score(y_unseen_test, y_unseen_pred)
unseen_f1 = f1_score(y_unseen_test, y_unseen_pred)
unseen_conf_matrix = confusion_matrix(y_unseen_test, y_unseen_pred)
end = datetime.now()

C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3150/3150 ━━━━━━━━━━━━━━━━━━━━ 2s 699us/step
350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 768us/step


C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3150/3150 ━━━━━━━━━━━━━━━━━━━━ 3s 907us/step
350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 761us/step


C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

3150/3150 ━━━━━━━━━━━━━━━━━━━━ 2s 666us/step
350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 679us/step


C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3150/3150 ━━━━━━━━━━━━━━━━━━━━ 2s 728us/step
350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 784us/step


C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3150/3150 ━━━━━━━━━━━━━━━━━━━━ 2s 690us/step
350/350 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step
875/875 ━━━━━━━━━━━━━━━━━━━━ 1s 682us/step


In [7]:

# Calculate the average confusion matrix
import numpy as np

# Initialize an array to store the sum of confusion matrices
confusion_matrix_sum = np.zeros((2, 2))  # Adjust the size if multi-class classification

# During the cross-validation loop, confusion matrices were already calculated
# Here, we'll sum up those matrices (assumes `confusion_matrices` contains all fold matrices)
for cm in confusion_matrices:
    confusion_matrix_sum += cm

# Compute the average confusion matrix
average_conf_matrix = confusion_matrix_sum / 10

# Display the average confusion matrix
print("Average Confusion Matrix:")
print(average_conf_matrix)

Average Confusion Matrix:
[[5472.5  141.4]
 [  44.1 5542. ]]


In [8]:
# Print the metrics for each fold
for i in range(10):
    print(f"Fold {i+1}:")
    print(f"Training Accuracy: {train_accuracies[i]}")
    print(f"Testing Accuracy: {test_accuracies[i]}")
    print(f"Precision: {precisions[i]}")
    print(f"Recall: {recalls[i]}")
    print(f"F1-Score: {f1_scores[i]}")
    print(f"Confusion Matrix:\n{confusion_matrices[i]}\n")

print(f"Average Training Accuracy: {sum(train_accuracies) / 10}")
print(f"Average Testing Accuracy: {sum(test_accuracies) / 10}")
print(f"Average Precision: {sum(precisions) / 10}")
print(f"Average recall: {sum(recalls) / 10}")
print(f"Average F1 Score: {sum(f1_scores) / 10}")

# Print the performance on the unseen data
print("Performance on Unseen Data:")
print(f"Accuracy: {unseen_accuracy}")
print(f"Precision: {unseen_precision}")
print(f"Recall: {unseen_recall}")
print(f"F1-Score: {unseen_f1}")
print(f"Confusion Matrix:\n{unseen_conf_matrix}")
print("excution time: ", end-start)
 

Fold 1:
Training Accuracy: 0.9833928571428572
Testing Accuracy: 0.9847321428571428
Precision: 0.9763074763074763
Recall: 0.9935702804072156
F1-Score: 0.984863238027795
Confusion Matrix:
[[5466  135]
 [  36 5563]]

Fold 2:
Training Accuracy: 0.9836507936507937
Testing Accuracy: 0.9823214285714286
Precision: 0.9742504409171076
Recall: 0.990674318507891
F1-Score: 0.9823937399964432
Confusion Matrix:
[[5478  146]
 [  52 5524]]

Fold 3:
Training Accuracy: 0.9837301587301587
Testing Accuracy: 0.9832142857142857
Precision: 0.9746131107633456
Recall: 0.9925624225252346
F1-Score: 0.9835058782242498
Confusion Matrix:
[[5407  146]
 [  42 5605]]

Fold 4:
Training Accuracy: 0.9833134920634921
Testing Accuracy: 0.9846428571428572
Precision: 0.9765124555160143
Recall: 0.9927641099855282
F1-Score: 0.9845712235378543
Confusion Matrix:
[[5540  132]
 [  40 5488]]

Fold 5:
Training Accuracy: 0.983343253968254
Testing Accuracy: 0.9847321428571428
Precision: 0.9778208061960922
Recall: 0.9919642857142857
F1-

In [9]:
# import pickle
# with open('ann_binary.pkl', 'wb') as file:
#     pickle.dump(model, file)

In [10]:
model.save('ann.h5')

In [11]:
model.save('ann.keras')